In [ ]:
!git clone https://github.com/chandraseta/spam-mails-detection

Cloning into 'spam-mails-detection'...
remote: Enumerating objects: 9202, done.
remote: Total 9202 (delta 0), reused 0 (delta 0), pack-reused 9202
Receiving objects: 100% (9202/9202), 9.24 MiB | 11.91 MiB/s, done.
Resolving deltas: 100% (4185/4185), done.
You can inspect what was checked out with 'git status'
and retry the checkout with 'git checkout -f HEAD'

^C


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Get fields out of .eml files

In [ ]:
import email.parser 
from email import policy
from email.parser import BytesParser
import glob
import os, sys, stat
import re
import shutil
import pandas as pd

In [ ]:
def ExtractFields (filename):
	''' 
	Extract fields from the .eml file.
	'''
	field_keys = ['Body', 'Delivery-Date', 'Return-Path', 'Received', 'Message-Id', 'Received', 'From', 'To', 'Subject', 'Date', 'X-M5MailerProjectID', 'Reply-To', 'MIME-Version', 'Content-Type', 'X-Return-Path', 'X-MDaemon-Deliver-To', 'X-Virus-Scanned', 'X-Status', 'X-Keywords']
	fields_dict = dict.fromkeys(field_keys, None)
	if not os.path.exists(filename): # dest path doesnot exist
		print("ERROR: input file does not exist:", filename)
		os.exit(1)
	msg = None
	try: 
		fp = open(filename, encoding="unicode_escape")
		msg = email.message_from_file(fp)
		fp.close()
	except UnicodeDecodeError: 
		try:
			fp = open(filename, encoding="utf-7")
			print("try utf-7")
			msg = email.message_from_file(fp)
			fp.close()
		except UnicodeDecodeError:
			try: 
				fp = open(filename, encoding="utf-8")
				print("try utf-8")
				msg = email.message_from_file(fp)
				fp.close()
			except UnicodeDecodeError:
				print("failed all 3")
				return None

	payload = msg.get_payload()
	if type(payload) == type(list()) :
		payload = payload[0] # only use the first part of payload
	sub = msg.get('subject')
	sub = str(sub)
	if type(payload) != type('') :
		payload = str(payload)
 
	fields_dict['Body'] = payload
	with open(filename, 'rb') as fp:
		fields = BytesParser(policy=policy.default).parse(fp)
	for f_key in fields.keys():
		if f_key in field_keys:
			fields_dict[f_key] = fields[f_key]
	return fields_dict

In [ ]:
data_dict = {}
# directory = '/content/spam-mails-detection/dataset/TR-mails/TR-raw'
directory = '/content/drive/MyDrive/6220 Project/SpamDatasets (1)/CSDMC2010_RAW/TRAINING'
for file_name in os.listdir(directory):
    if file_name.endswith(".eml"):
        start = file_name.find("TRAIN_") + len("TRAIN_")
        end = file_name.find(".eml")
        train_sample_num = str(int(file_name[start:end]))
        file_path = os.path.join(directory, file_name)
        print('Extracting from..... {}', file_name)
        fields_dict = ExtractFields(file_path)
        data_dict[train_sample_num] = fields_dict
    else:
        continue

Extracting from..... {} TRAIN_02256.eml
Extracting from..... {} TRAIN_03230.eml
Extracting from..... {} TRAIN_03897.eml
Extracting from..... {} TRAIN_00022.eml
Extracting from..... {} TRAIN_04135.eml
Extracting from..... {} TRAIN_04036.eml
Extracting from..... {} TRAIN_03950.eml
Extracting from..... {} TRAIN_02327.eml
Extracting from..... {} TRAIN_03946.eml
Extracting from..... {} TRAIN_00876.eml
Extracting from..... {} TRAIN_03872.eml
Extracting from..... {} TRAIN_03368.eml
Extracting from..... {} TRAIN_02552.eml
Extracting from..... {} TRAIN_01250.eml
Extracting from..... {} TRAIN_04195.eml
Extracting from..... {} TRAIN_01751.eml
Extracting from..... {} TRAIN_02487.eml
Extracting from..... {} TRAIN_00817.eml
Extracting from..... {} TRAIN_00053.eml
Extracting from..... {} TRAIN_00884.eml
Extracting from..... {} TRAIN_02164.eml
Extracting from..... {} TRAIN_02713.eml
Extracting from..... {} TRAIN_03817.eml
Extracting from..... {} TRAIN_02177.eml
Extracting from..... {} TRAIN_01251.eml


KeyboardInterrupt: ignored

In [ ]:
print(len(data_dict.keys()))

4327


In [ ]:
data_dict['1345']

{'Body': 'use Perl Daily Newsletter\n\nIn this issue:\n    * "Perl 6: Right Here, Right Now" slides ava\n\n+--------------------------------------------------------------------+\n| "Perl 6: Right Here, Right Now" slides ava                         |\n|   posted by gnat on Friday September 13, @12:01 (news)             |\n|   http://use.perl.org/article.pl?sid=02/09/13/162209               |\n+--------------------------------------------------------------------+\n\n[0]gnat writes "The wonderful Leon Brocard has released the slides from\nhis lightning talk to the London perlmongers, [1]Perl 6: Right Here,\nRight Now, showing the current perl6 compiler in action."\n\nDiscuss this story at:\n    http://use.perl.org/comments.pl?sid=02/09/13/162209\n\nLinks:\n    0. mailto:gnat@oreilly.com\n    1. http://astray.com/perl6_now/\n\n\n\nCopyright 1997-2002 pudge.  All rights reserved.\n\n\n======================================================================\n\nYou have received this message be



# Get ground truth labels in the form <sample_num, label>
0 = not spam, 1 = spam



In [ ]:
from csv import reader
# labels_file_path = '/content/spam-mails-detection/dataset/spam-mail.tr.label'
labels_file_path = '/content/drive/MyDrive/6220 Project/SpamDatasets (1)/CSDMC2010_RAW/SPAMTrain.label'
with open(labels_file_path, 'r') as read_obj:
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        # row variable is a list that represents a row in csv
        label, train_file_name = row[0].split(' ')
        start = train_file_name.find("TRAIN_") + len("TRAIN_")
        end = train_file_name.find(".eml")
        sample_num = str(int(train_file_name[start:end]))
        print('sample num: {}, label: {}'.format(sample_num, label))
        fields_dict = data_dict[sample_num]
        if fields_dict is None: 
          continue
        fields_dict['spam_label'] = label

sample num: 0, label: 0
sample num: 1, label: 0
sample num: 2, label: 1


KeyError: ignored

In [ ]:
data_dict['1345']

{'Body': 'use Perl Daily Newsletter\n\nIn this issue:\n    * "Perl 6: Right Here, Right Now" slides ava\n\n+--------------------------------------------------------------------+\n| "Perl 6: Right Here, Right Now" slides ava                         |\n|   posted by gnat on Friday September 13, @12:01 (news)             |\n|   http://use.perl.org/article.pl?sid=02/09/13/162209               |\n+--------------------------------------------------------------------+\n\n[0]gnat writes "The wonderful Leon Brocard has released the slides from\nhis lightning talk to the London perlmongers, [1]Perl 6: Right Here,\nRight Now, showing the current perl6 compiler in action."\n\nDiscuss this story at:\n    http://use.perl.org/comments.pl?sid=02/09/13/162209\n\nLinks:\n    0. mailto:gnat@oreilly.com\n    1. http://astray.com/perl6_now/\n\n\n\nCopyright 1997-2002 pudge.  All rights reserved.\n\n\n======================================================================\n\nYou have received this message be

In [ ]:
train_df = pd.DataFrame(data_dict)
train_df = train_df.transpose()

In [ ]:
train_df.head()

,Body,Delivery-Date,Return-Path,Received,Message-Id,From,To,Subject,Date,X-M5MailerProjectID,Reply-To,MIME-Version,Content-Type,X-Return-Path,X-MDaemon-Deliver-To,X-Virus-Scanned,X-Status,X-Keywords,spam_label
2256,"Just got this ... I was just reading mail, but...",None,<exmh-workers-admin@example.com>,from localhost (localhost [127.0.0.1])\tby pho...,<16323.1030043119@munnari.OZ.AU>,Robert Elz <kre@munnari.OZ.AU>,exmh-workers@example.com,Anolther sequence related traceback,"Fri, 23 Aug 2002 02:05:19 +0700",None,None,1.0,"text/plain; charset=""us-ascii""",None,None,None,None,None,1
3230,Today's Headlines from The Register\n---------...,None,<update@list.theregister.co.uk>,from webnote.net (mail.webnote.net [193.120.21...,<E17WSYl-0007dp-0C@list.theregister.co.uk>,update@list.theregister.co.uk,Reg Subscribers <update@list.theregister.co.uk>,Reg Headlines Monday July 22,"Mon, 22 Jul 2002 03:00:01 +0100",None,None,None,None,None,None,None,None,None,1
3897,"Content-Type: text/plain;\n\tcharset=""Windows-...",None,<declamation838@riolink.com>,from mail3.csmining.org (localhost [127.0.0.1]...,None,Earlene Decker <declamation838@riolink.com>,hibody@csmining.org,[SPAM] Health problems? Forget about them wit...,"Wed, 14 Oct 2009 13:56:49 +0100",None,None,1.0,"multipart/alternative; boundary=""----=_NextPar...",None,None,None,None,None,0
22,"\n<html><body>\n<center>\n<table bgcolor=3D""#0...",None,<iteachabc@msn.com>,from localhost (localhost [127.0.0.1])\tby pho...,None,Local Guest <iteachabc@msn.com>,Undisclosed.Recipients@webnote.net,Look What Sandy is Doing in her DORM!!,"Fri, 23 Aug 2002 10:50:21 -1600",None,iteachabc@msn.com,1.0,"text/html; charset=""iso-8859-1""",None,None,None,None,None,0
4135,"On Tue, Jul 30, 2002 at 03:47:40PM +0100 or so...",None,<ilug-admin@linux.ie>,from localhost (localhost [127.0.0.1])\tby pho...,<20020731175701.B4779@Hobbiton.cod.ie>,Conor Daly <conor.daly@oceanfree.net>,ilug@linux.ie,Re: [ILUG] Installing lilo on another disk.,"Wed, 31 Jul 2002 17:57:01 +0100",None,None,1.0,"text/plain; charset=""us-ascii""",None,None,None,None,None,1


In [ ]:
train_df.to_csv('/content/drive/MyDrive/6220 Project/SpamDatasets/CSDMC2010_RAW/Neha_dataframes/unprocessed_train_df.csv')

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/6220 Project/SpamDatasets/CSDMC2010_RAW/Neha_dataframes/unprocessed_train_df.csv')

# Extract Fields That Matter (Subject + Body)

In [ ]:
unprocessed_df = pd.read_csv('/content/drive/MyDrive/6220 Project/SpamDatasets (1)/CSDMC2010_RAW/Neha_dataframes/unprocessed_train_df.csv')

In [ ]:
unprocessed_df.head(2)

,Unnamed: 0,Body,Delivery-Date,Return-Path,Received,Message-Id,From,To,Subject,Date,X-M5MailerProjectID,Reply-To,MIME-Version,Content-Type,X-Return-Path,X-MDaemon-Deliver-To,X-Virus-Scanned,X-Status,X-Keywords,spam_label
0,2256,"Just got this ... I was just reading mail, but...",NaN,<exmh-workers-admin@example.com>,from localhost (localhost [127.0.0.1])\tby pho...,<16323.1030043119@munnari.OZ.AU>,Robert Elz <kre@munnari.OZ.AU>,exmh-workers@example.com,Anolther sequence related traceback,"Fri, 23 Aug 2002 02:05:19 +0700",NaN,NaN,1.0,"text/plain; charset=""us-ascii""",NaN,NaN,NaN,NaN,NaN,1.0
1,3230,Today's Headlines from The Register\n---------...,NaN,<update@list.theregister.co.uk>,from webnote.net (mail.webnote.net [193.120.21...,<E17WSYl-0007dp-0C@list.theregister.co.uk>,update@list.theregister.co.uk,Reg Subscribers <update@list.theregister.co.uk>,Reg Headlines Monday July 22,"Mon, 22 Jul 2002 03:00:01 +0100",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [ ]:
unprocessed_df = unprocessed_df[['Subject', 'Body', 'spam_label']]

In [ ]:
unprocessed_df.isna().sum()

Subject       7
Body          1
spam_label    1
dtype: int64

In [ ]:
def convert_NaN_to_str(text):
  if text == 'nan':
    return ''
  return text

In [ ]:
unprocessed_df = unprocessed_df[unprocessed_df['spam_label'].notna()]
unprocessed_df['Subject'] = unprocessed_df['Subject'].astype(str)
unprocessed_df['Subject'] = unprocessed_df['Subject'].apply(convert_NaN_to_str)
unprocessed_df.isna().sum()

Subject       0
Body          0
spam_label    0
dtype: int64

In [ ]:
unprocessed_df['Content'] = unprocessed_df['Subject'] + ' ' + unprocessed_df['Body']
unprocessed_df['spam_label'] = unprocessed_df['spam_label'].astype(int) 
unprocessed_df.head(2)

,Subject,Body,spam_label,Content
0,Anolther sequence related traceback,"Just got this ... I was just reading mail, but...",1,Anolther sequence related traceback Just got t...
1,Reg Headlines Monday July 22,Today's Headlines from The Register\n---------...,1,Reg Headlines Monday July 22 Today's Headlines...


In [ ]:
unprocessed_df = unprocessed_df[['Content', 'spam_label']]

In [ ]:
unprocessed_df.to_csv('/content/drive/MyDrive/6220 Project/SpamDatasets (1)/CSDMC2010_RAW/Neha_dataframes/subject-body-unprocessed.csv')

# Pre-processing & Lemmatization



In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/6220 Project/SpamDatasets (1)/CSDMC2010_RAW/Neha_dataframes/subject-body-unprocessed.csv')

In [ ]:
import glob
import nltk
import re, unicodedata

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
!pip install contractions
import contractions

In [ ]:
def denoise_data(content):
    content_processed = str(content)
    content_processed = BeautifulSoup(content_processed, 'html.parser').text
    content_processed = contractions.fix(content_processed)
    return content_processed

In [ ]:
train_df['Content'] = train_df['Content'].apply(denoise_data)
# TODO: add to subject

In [ ]:
def normalize_data(unprocessed_content):
    content = str(unprocessed_content)

    # replace escaped sequences
    content = content.replace('\n', ' ')
    content = content.replace('\r', ' ')

    words = nltk.word_tokenize(content) 
    
    # Remove long words
    new_words = []
    for word in words:
        if len(word) <= 20:
            new_words.append(word)
    words = new_words

    # Remove Non-ASCII words
    new_words = []
    for word in words:
      new_word = unicodedata.normalize('NFKD',  word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
      new_words.append(new_word)
    words = new_words 

    # Convert to lowercase
    new_words = []
    for word in words:
      new_word = word.lower()
      new_words.append(new_word)
    words = new_words  

    # Remove punctuations
    new_words = []
    for word in words:
      new_word = re.sub(r'[^\w\s]', ' ', word)
      if new_word != '' or new_word != ' ' or new_word != '  ':
        new_words.append(new_word.strip())
    words = new_words

    # Remove numbers
    new_words = []
    for word in words:
      new_word = re.sub(r'\d+', '', word)
      if new_word != '':
        new_words.append(new_word)
    words = new_words

    # Remove stopwords
    new_words = []
    for word in words:
      if word not in nltk.corpus.stopwords.words('english'):
        new_words.append(word)
    words = new_words 

    # Lemmatize
    new_words = []
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
      lemma = lemmatizer.lemmatize(word, pos='v')
      lemmas.append(lemma)
      new_lemmas = [x.strip() for x in lemmas]
      lemmas = [x for x in new_lemmas if x != '']
    str_lemmas = ' '.join(str(x) for x in lemmas)

    return str_lemmas

In [ ]:
train_df['Content'] = train_df['Content'].apply(normalize_data)

In [ ]:
train_df.to_csv('/content/drive/MyDrive/6220 Project/SpamDatasets (1)/CSDMC2010_RAW/Neha_dataframes/subject-body-processed.csv')